In [65]:
# Imports for building the model
import keras
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K #not sure what this is for

# Training parameters
batch_size = 128
epochs = 5

# Loading the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Storing number of rows + columns in an image
img_rows = x_train.shape[1]
img_cols = x_train.shape[2]

# Keras needs 4D image, so need to add a fourth dimension. Data shape goes from: (60000, 28, 28) -> (60000, 28, 28, 1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# Storing shape of a single image
input_shape = (img_rows, img_cols, 1)

# Change image to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0-255) -> (0-1)
x_train /= 255
x_test /= 255

print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Hot one encoding the classes
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Counting number of columns in hot one encoded matrix
num_classes = y_test.shape[1]

num_pixels = x_train.shape[1] * x_train.shape[2]

# Creating model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

print(model.summary())

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test,y_test))

score = model.evaluate(x_test, y_test, verbose=0)

print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])

x_train shape:  (60000, 28, 28, 1)
60000 train samples
10000 test samples
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
______________________________________________

In [66]:
model.save("Desktop\DeepLearning\MNIST-Handwriting-Classifier.h5")

In [68]:
import cv2
import numpy as np
from keras.datasets import mnist
from keras.models import load_model

classifier = load_model('Desktop\DeepLearning\MNIST-Handwriting-Classifier.h5')

# Loading MNIST dataset into variables
(x_train, y_train), (x_test, y_test) = mnist.load_data()

def draw_test(name, pred, input_im):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(input_im, 0, 0, 0, imageL.shape[0], cv2.BORDER_CONSTANT, value=BLACK)
    expanded_image = cv2.cvtColor(expanded_image, cv2.COLOR_GRAY2BGR)
    # drawing the number that is predicted by the model
    cv2.putText(expanded_image, str(pred), (152, 70), cv2.FONT_HERSHEY_COMPLEX_SMALL, 4, (0, 255, 0), 2)
    cv2.imshow(name, expanded_image)

for i in range(0, 10):
    rand = np.random.randint(0, len(x_test))
    input_im = x_test[rand]
    
    imageL = cv2.resize(input_im, None, fx=4, fy=4, interpolation= cv2.INTER_CUBIC)
    input_im = input_im.reshape(1, 28, 28, 1)
    
    # Getting the prediction
    res = str(classifier.predict_classes(input_im, 1, verbose=0)[0])
    draw_test("Prediction", res, imageL)
    cv2.waitKey(0)

cv2.destroyAllWindows()